# Head First - Testing

In [1]:
!wget https://www.stud.fit.vutbr.cz/~ifajcik/r2d2/NQ/irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip

--2021-04-20 07:52:54--  https://www.stud.fit.vutbr.cz/~ifajcik/r2d2/NQ/irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip
Resolving www.stud.fit.vutbr.cz (www.stud.fit.vutbr.cz)... 147.229.176.14, 2001:67c:1220:8b0::93e5:b00e
Connecting to www.stud.fit.vutbr.cz (www.stud.fit.vutbr.cz)|147.229.176.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406363167 (388M) [application/zip]
Saving to: ‘irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip’

irrelevant_doc_cls_ 100%[===================>] 387.54M  4.28MB/s    in 1m 59s  

2021-04-20 07:54:54 (3.27 MB/s) - ‘irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip’ saved [406363167/406363167]



In [2]:
!unzip irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip

Archive:  irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip
  inflating: irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt  


In [3]:
!git clone https://github.com/KNOT-FIT-BUT/scalingQA.git

Cloning into 'scalingQA'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 153 (delta 21), reused 149 (delta 21), pack-reused 0
Receiving objects: 100% (153/153), 5.72 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [4]:
!mkdir index_pruning

In [5]:
cp -r scalingQA/scalingqa/index_pruning/* index_pruning/

In [6]:
!pip install -q transformers==4.3.3 torchtext==0.4.0

     |████████████████████████████████| 1.9MB 4.3MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 3.3MB 21.6MB/s 
     |████████████████████████████████| 901kB 47.0MB/s 


In [7]:
import torchtext
import torch

In [18]:
path = "irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
model = torch.load(path, map_location=device)

In [4]:
model.eval()

TransformerBinaryClassifier(
  (transformer): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator",use_fast=True)

In [15]:
from torchtext import data

INPUT_field = data.Field(use_vocab=False, batch_first=True, sequential=True, pad_token=tokenizer.pad_token_id)
SEGMENT_field = PADDING_field = data.Field(use_vocab=False, batch_first=True, sequential=True, pad_token=0)
    
def prepare_batch(raw_batch: dict, max_len: int = 128, include_title=True):
    inputs = []
    input_segments = []
    input_paddings = []
    title_batch, psg_batch = raw_batch["title"], raw_batch["psg"]
    assert len(title_batch) == len(psg_batch)
    for title, passage in zip(title_batch, psg_batch):
        if include_title:
            preprocessed = tokenizer.encode_plus(title, passage, add_special_tokens=True,
                                                      return_token_type_ids=True, truncation=True,
                                                      max_length=max_len)
        else:
            preprocessed = tokenizer.encode_plus(passage, add_special_tokens=True,
                                                      return_token_type_ids=True, truncation=True,
                                                      max_length=max_len)
        input_ids, segment_mask = preprocessed['input_ids'], preprocessed['token_type_ids']
        inputs.append(input_ids)
        input_segments.append(segment_mask)
        input_paddings.append([1] * len(input_ids))

    lt = lambda x: torch.LongTensor(x).to(device)

    inputs = INPUT_field.pad(inputs)
    segments = SEGMENT_field.pad(input_segments)
    input_masks = PADDING_field.pad(input_paddings)

    return lt(inputs), lt(segments), lt(input_masks)

In [16]:
raw_batch_dict = {
    "title" : ['Narendra Modi', "Rahul Gandhi"],
    "psg": ["Hi I am NM from India", "Hi I am Pappu from India"]
}

In [19]:
inputs, segments, input_masks = prepare_batch(raw_batch_dict)

In [9]:
from transformers import ElectraConfig
config = ElectraConfig.from_pretrained("google/electra-base-discriminator")

In [10]:
config

ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [37]:
scores = model(input_ids=inputs, token_type_ids=segments, attention_mask=input_masks)
scores

tensor([3.3163, 4.9682], grad_fn=<SqueezeBackward1>)

In [8]:
config = {
    "tokenizer_type": "google/electra-base-discriminator",
    "model_type": "google/electra-base-discriminator",

    "data_cache_dir": '.data/nq_corpus_pruning',
    "training_data": "train.jsonl",
    "validation_data": "val.jsonl",
    "test_data": "test.jsonl",

    "x-negatives": 2,  # how many times more negative passages are in the training set

    "scheduler": "linear",

    # hyperparams for BERT
    "epochs": 2,
    "batch_size": 6,
    "true_batch_size": 12,
    "max_grad_norm": 1.0,
    "weight_decay": 0.0,
    "learning_rate": 3e-05,
    "adam_eps": 1e-08,
    "warmup_steps": 0,
    "cls_dropout": 0.1,

    "validation_batch_size": 8,
    "validate_update_steps": 2500,

    "cache_dir": ".Transformers_cache",
    "use_title": True,

    "save_dir": ".saved",
    "multi_gpu": False,
    "test_only": False,

    "model_to_validate": ""
}

In [11]:
from index_pruning.models.transformer_binary_cls import *

In [12]:
model = TransformerBinaryClassifier(config)

In [23]:
model_2 = torch.load(path, map_location=device)

In [34]:
model.load_state_dict(model_2.state_dict())

<All keys matched successfully>

# Clean Code - Actual (Use this code only)

In [1]:
!wget https://www.stud.fit.vutbr.cz/~ifajcik/r2d2/NQ/irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip

--2021-04-21 04:23:58--  https://www.stud.fit.vutbr.cz/~ifajcik/r2d2/NQ/irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip
Resolving www.stud.fit.vutbr.cz (www.stud.fit.vutbr.cz)... 147.229.176.14, 2001:67c:1220:8b0::93e5:b00e
Connecting to www.stud.fit.vutbr.cz (www.stud.fit.vutbr.cz)|147.229.176.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406363167 (388M) [application/zip]
Saving to: ‘irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip’

irrelevant_doc_cls_ 100%[===================>] 387.54M  2.71MB/s    in 1m 55s  

2021-04-21 04:25:54 (3.36 MB/s) - ‘irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip’ saved [406363167/406363167]



In [2]:
!unzip irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip

Archive:  irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt.zip
  inflating: irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt  


In [3]:
!pip install -q transformers==4.3.3 torchtext==0.4.0

     |████████████████████████████████| 1.9MB 4.4MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 901kB 17.6MB/s 
     |████████████████████████████████| 3.3MB 27.5MB/s 


In [4]:
!mkdir index_pruning
!git clone https://github.com/KNOT-FIT-BUT/scalingQA.git

Cloning into 'scalingQA'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 153 (delta 21), reused 149 (delta 21), pack-reused 0
Receiving objects: 100% (153/153), 5.72 MiB | 19.13 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [5]:
cp -r scalingQA/scalingqa/index_pruning/* index_pruning/

In [6]:
import torchtext
import torch

In [7]:
path = "irrelevant_doc_cls_google_electra-base-discriminator_acc_0.9049_2020-12-26_23:51.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [8]:
from index_pruning.models.transformer_binary_cls import TransformerBinaryClassifier

In [9]:
config = {
    "tokenizer_type": "google/electra-base-discriminator",
    "model_type": "google/electra-base-discriminator",

    "data_cache_dir": '.data/nq_corpus_pruning',
    "training_data": "train.jsonl",
    "validation_data": "val.jsonl",
    "test_data": "test.jsonl",

    "x-negatives": 2,  # how many times more negative passages are in the training set

    "scheduler": "linear",

    # hyperparams for BERT
    "epochs": 2,
    "batch_size": 6,
    "true_batch_size": 12,
    "max_grad_norm": 1.0,
    "weight_decay": 0.0,
    "learning_rate": 3e-05,
    "adam_eps": 1e-08,
    "warmup_steps": 0,
    "cls_dropout": 0.1,

    "validation_batch_size": 8,
    "validate_update_steps": 2500,

    "cache_dir": ".Transformers_cache",
    "use_title": True,

    "save_dir": ".saved",
    "multi_gpu": False,
    "test_only": False,

    "model_to_validate": ""
}

model = TransformerBinaryClassifier(config)

In [10]:
model_weights = torch.load(path, map_location=device)
model.load_state_dict(model_weights.state_dict())

<All keys matched successfully>

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator",use_fast=True)

In [12]:
from torchtext import data

INPUT_field = data.Field(use_vocab=False, batch_first=True, sequential=True, pad_token=tokenizer.pad_token_id)
SEGMENT_field = PADDING_field = data.Field(use_vocab=False, batch_first=True, sequential=True, pad_token=0)
    
def prepare_batch(raw_batch: dict, max_len: int = 128, include_title=True):
    inputs = []
    input_segments = []
    input_paddings = []
    title_batch, psg_batch = raw_batch["title"], raw_batch["psg"]
    assert len(title_batch) == len(psg_batch)
    for title, passage in zip(title_batch, psg_batch):
        if include_title:
            preprocessed = tokenizer.encode_plus(title, passage, add_special_tokens=True,
                                                      return_token_type_ids=True, truncation=True,
                                                      max_length=max_len)
        else:
            preprocessed = tokenizer.encode_plus(passage, add_special_tokens=True,
                                                      return_token_type_ids=True, truncation=True,
                                                      max_length=max_len)
        input_ids, segment_mask = preprocessed['input_ids'], preprocessed['token_type_ids']
        inputs.append(input_ids)
        input_segments.append(segment_mask)
        input_paddings.append([1] * len(input_ids))

    lt = lambda x: torch.LongTensor(x).to(device)

    inputs = INPUT_field.pad(inputs)
    segments = SEGMENT_field.pad(input_segments)
    input_masks = PADDING_field.pad(input_paddings)

    return lt(inputs), lt(segments), lt(input_masks)

In [13]:
# Example dict (should be in batch i.e. 32 or 64)
"""raw_batch_dict = {
    "title" : ['Narendra Modi', "Rahul Gandhi"],
    "psg": ["Hi I am NM from India", "Hi I am Pappu from India"]
}"""

raw_batch_dict = {
    "title" : ['William Henry Gates',
               'William Henry Gates'],
    "psg": ["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, author, landowner and philanthropist. He is a co-founder of Microsoft Corporation.",
            "During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014.[citation needed] He is considered one of the best known entrepreneurs of the microcomputer revolution of the 1970s and 1980s. Gates is also co-founder of TerraPower that is developing next generation traveling-wave reactor nuclear power plants."]
}


In [14]:
inputs, segments, input_masks = prepare_batch(raw_batch_dict)

In [15]:
scores = model(input_ids=inputs, token_type_ids=segments, attention_mask=input_masks)
scores

tensor([-4.8244,  4.7094], grad_fn=<SqueezeBackward1>)

In [ ]:
# Take Threshold > -2.7950857 for Keeping 1700000 indices

# Reference: https://github.com/KNOT-FIT-BUT/scalingQA/blob/main/scalingqa/index_pruning/inference/get_pruning_index.ipynb


In [16]:
t = -2.7950857
indices_to_keep =(scores<t).nonzero()
indices_to_keep

tensor([[0]])